#### First we import the neccesary libraries

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import joblib

#### Reading and preprocessing the dataset

In [2]:
df = pd.read_csv("data/cocoset.csv").dropna()
df.columns = df.columns.str.lstrip().str.strip().str.replace(' ','_')

In [3]:
%%capture
scaler = MinMaxScaler()
for col in df.columns:
    x = col+'x'
    y = col+'y'
    z = col+'z'
    #splitting
    df[[x, y,z]] = df[col].str.strip('()').str.split(',', expand=True)
    #converting to number
    df[x] = df[x].apply(pd.to_numeric, errors='coerce')
    df[y] = df[y].apply(pd.to_numeric, errors='coerce')
    #scaling to range
    df[x] = scaler.fit_transform(df[[x]])
    df[y] = scaler.fit_transform(df[[y]])

    #dropping redundant cols
    df.drop([col], inplace=True, axis=1)
    df.drop([z],inplace=True,axis=1)
df.dropna()

#### Reordering and renaming column

In [ ]:
old_columns = joblib.load("pkl_files/column_names.pkl")
mapping = joblib.load("pkl_files/mapping.pkl")

In [5]:
reordered_columns = []
for name in old_columns:
    part = name.split('_(')[1].rstrip(')')
    axis = name[0]
    colname = mapping[part][0] if axis == 'x' else mapping[part][1]
    reordered_columns.append(colname)

df = df[reordered_columns]
df.columns = old_columns

#### Predicting poses

In [ ]:
cla = joblib.load("pkl_files/rf_pretrained.pkl")
y_pred = pd.DataFrame(cla.predict(df))
poses = y_pred.replace([0,1],['sitting','standing'])


In [7]:
poses.head()

,0
0,standing
1,standing
2,standing
3,standing
4,standing
